In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/seed-iv/Channel Order.xlsx
/kaggle/input/seed-iv/ReadMe.txt
/kaggle/input/seed-iv/eye_feature_smooth/2/4_20151118.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/14_20151208.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/5_20160413.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/3_20151018.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/6_20150511.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/15_20150514.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/10_20151021.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/12_20150804.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/13_20151125.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/11_20150921.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/8_20151110.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/9_20151119.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/2_20150920.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/7_20150717.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/1_20161125.mat
/kaggle/input/seed-iv/eye_feature_smooth/3/14_2015

In [1]:
pip install braindecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 4.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 9.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import tensorflow as tf

import scipy
import os
from os import listdir
from scipy.io import loadmat
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 
from tensorflow.keras import regularizers
from tensorflow.keras import activations

In [3]:
import os
import numpy as np
from scipy.io import loadmat

# Correct paths for Kaggle dataset
directories = ["/kaggle/input/seed-iv/eeg_feature_smooth/{}".format(i+1) for i in range(3)] 
print(directories)

# EEG channel coordinates
channel_coords = [
    ['0', '0', 'AF3', 'FP1', 'FPZ', 'FP2', 'AF4', '0', '0'], 
    ['F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'], 
    ['FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'], 
    ['T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'], 
    ['TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'], 
    ['P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'], 
    ['0', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '0'], 
    ['0', '0', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '0', '0']
]

channel_list = [
    'FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 
    'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 
    'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 
    'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 
    'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 
    'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 
    'O1', 'OZ', 'O2', 'CB2'
]

# Verifying the length of channel coordinates
print(len(channel_coords), len(channel_coords[0]))

# Creating a dictionary to map channel names to coordinates
coord_dict = {}
for n in range(len(channel_list)):
    for i, l in enumerate(channel_coords):
        for j, x in enumerate(l):
            if channel_list[n] == x:
                coord_dict[n] = (i, j)
print(coord_dict)

# Initialize the array with the correct shape
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories), len(os.listdir(directories[0])), n, 4, 8, 9, 5, 64)) 

# Iterate over directories to load and process data
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(os.path.join(dire, file)) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
        for j in range(n):
            for k, key in enumerate(perSample):
                # Transpose and pad the sample
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0, 2, 1))
                sample = np.pad(sample, [(0, 0), (0, 0), (0, 64 - sample.shape[2])])
                # Fill the array based on channel locations
                for l, channel in enumerate(sample):
                    array[h][i][j][k][coord_dict[l][0]][coord_dict[l][1]] = channel

# Output the shape of the final array
print(array.shape)


['/kaggle/input/seed-iv/eeg_feature_smooth/1', '/kaggle/input/seed-iv/eeg_feature_smooth/2', '/kaggle/input/seed-iv/eeg_feature_smooth/3']
8 9
{0: (0, 3), 1: (0, 4), 2: (0, 5), 3: (0, 2), 4: (0, 6), 5: (1, 0), 6: (1, 1), 7: (1, 2), 8: (1, 3), 9: (1, 4), 10: (1, 5), 11: (1, 6), 12: (1, 7), 13: (1, 8), 14: (2, 0), 15: (2, 1), 16: (2, 2), 17: (2, 3), 18: (2, 4), 19: (2, 5), 20: (2, 6), 21: (2, 7), 22: (2, 8), 23: (3, 0), 24: (3, 1), 25: (3, 2), 26: (3, 3), 27: (3, 4), 28: (3, 5), 29: (3, 6), 30: (3, 7), 31: (3, 8), 32: (4, 0), 33: (4, 1), 34: (4, 2), 35: (4, 3), 36: (4, 4), 37: (4, 5), 38: (4, 6), 39: (4, 7), 40: (4, 8), 41: (5, 0), 42: (5, 1), 43: (5, 2), 44: (5, 3), 45: (5, 4), 46: (5, 5), 47: (5, 6), 48: (5, 7), 49: (5, 8), 50: (6, 1), 51: (6, 2), 52: (6, 3), 53: (6, 4), 54: (6, 5), 55: (6, 6), 56: (6, 7), 57: (7, 2), 58: (7, 3), 59: (7, 4), 60: (7, 5), 61: (7, 6)}
/kaggle/input/seed-iv/eeg_feature_smooth/1
/kaggle/input/seed-iv/eeg_feature_smooth/2
/kaggle/input/seed-iv/eeg_feature_sm

In [5]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}

y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)

print(y.shape)

(1080,)


In [6]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])
print(_X.shape)

(1080, 4, 8, 9, 5, 64)


In [7]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


In [8]:
from braindecode.classifier import EEGClassifier
from sklearn.model_selection import train_test_split
import torch
import numpy as np
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [9]:
from braindecode.classifier import EEGClassifier
from braindecode.models import ShallowFBCSPNet

from braindecode.models import Deep4Net

deep4net = Deep4Net(
    n_outputs=4,               # Số lớp đầu ra
    n_chans=64,                # Số kênh EEG
    n_times=1440,              # Số mẫu thời gian
    drop_prob=0.5,             # Dropout
    sfreq=256, 
    final_conv_length='auto',
    n_filters_time=32,
    n_filters_spat=32,
    filter_time_length=10,
    n_filters_2=64,
    filter_length_2=3,
    n_filters_3=128,
    filter_length_3=3,
    n_filters_4=256,
    filter_length_4=3,
    batch_norm=True,
)

/opt/conda/lib/python3.10/site-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [10]:
from braindecode.classifier import EEGClassifier
from torch.optim import Adam

# Định nghĩa bộ phân loại
classifier = EEGClassifier(
    module=deep4net,
    criterion=torch.nn.CrossEntropyLoss,  # Hàm mất mát (phân loại chéo)
    optimizer=Adam,                       # Bộ tối ưu hóa (Adam)
    optimizer__lr=0.001,                  # Learning rate
    max_epochs=100,  
    batch_size=1,#                       # Kích thước batch
    device='cuda' if torch.cuda.is_available() else 'cpu'# Sử dụng GPU nếu khả dụng
)

In [11]:
classifier.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.0771       0.2486        3.5435  4.9541
      2        2.2584       0.2543        4.5397  3.4277
      3        1.8471       0.2948       42.3299  3.2361
      4        1.6121       0.2428        4.7042  3.5944
      5        1.5274       0.2428        1.6015  3.6016
      6        1.4741       0.2428       21.6348  3.3905
      7        1.4127       0.2486        2.6987  3.5500
      8        1.3677       0.2543       18.5317  3.4550
      9        1.3246       0.2601        1.4349  3.4601
     10        1.2289       0.2254       62.6787  3.4886
     11        1.2124       0.2543        2.0666  3.4316
     12        1.1321       0.2428        1.7668  3.3803
     13        1.0937       0.2659        1.6942  3.6792
     14        1.0146       0.2486        9.9057  3.5020
     15        0.9570       0.2601        1.4526  3.5030
     16        0.9297       0.2

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  Deep4Net (Deep4Net)                      [1, 64, 1440]             [1, 4]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 64, 1440]             [1, 64, 1440, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 64, 1440, 1]          [1, 1, 1440, 64]          --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1440, 64]          [1, 32, 1431, 1]          65,888                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 32, 1431, 1]          [1, 32, 1431, 1]          64                        --
  ├─Expression (conv_nonlin): 1-5 

In [12]:
from braindecode.models import EEGITNet
eegnetv4 = EEGITNet(
    n_outputs=4,               # Số lớp đầu ra
    n_chans=64,                # Số kênh EEG
    n_times=1440,              # Số mẫu thời gian
    drop_prob=0.5,             # Dropout
    sfreq=256, 
)

classifier_eegnet = EEGClassifier(
    module=deep4net,
    criterion=torch.nn.CrossEntropyLoss,  # Hàm mất mát (phân loại chéo)
    optimizer=Adam,                       # Bộ tối ưu hóa (Adam)
    optimizer__lr=0.001,                  # Learning rate
    max_epochs=100,  
    batch_size=1,#                       # Kích thước batch
    device='cuda' if torch.cuda.is_available() else 'cpu'  # Sử dụng GPU nếu khả dụng
)


/opt/conda/lib/python3.10/site-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [13]:
classifier_eegnet.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.1473       0.2486        6.6673  3.3845
      2        0.1563       0.2543        6.8374  3.6306
      3        0.1045       0.2775       10.0997  3.4566
      4        0.1468       0.2659        2.5171  3.4981
      5        0.2135       0.2890       22.4615  3.3105
      6        0.0749       0.2659        2.5502  3.2626
      7        0.1241       0.2659        3.6427  3.3065
      8        0.1122       0.3526       99.1973  3.5053
      9        0.1064       0.2486        5.8413  3.5715
     10        0.1641       0.2428       18.8023  3.5571
     11        0.1075       0.2601        3.2687  3.5288
     12        0.1215       0.2543        4.1485  3.4149
     13        0.1039       0.2486        5.7451  3.4482
     14        0.0803       0.2659        3.3125  3.4879
     15        0.1690       0.2775       25.1124  3.3129
     16        0.1248       0.4

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  Deep4Net (Deep4Net)                      [1, 64, 1440]             [1, 4]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 64, 1440]             [1, 64, 1440, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 64, 1440, 1]          [1, 1, 1440, 64]          --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1440, 64]          [1, 32, 1431, 1]          65,888                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 32, 1431, 1]          [1, 32, 1431, 1]          64                        --
  ├─Expression (conv_nonlin): 1-5 